In [1]:
#https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [2]:
#1. Dataset preparation3. Model Building
# load the dataset
data = open('data/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [3]:
trainDF.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [4]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


In [5]:
#2.1 Count Vectors as features
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)


In [6]:
#2.2 TF-IDF Vectors as features
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

In [7]:
#2.3 Word Embeddings
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [8]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.0897      0.016      -0.0571     ...  0.1559     -0.0254
  -0.0259    ]
 [-0.0314      0.0149     -0.0205     ...  0.098       0.0893
   0.0148    ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.0897     -0.0682      0.0726     ...  0.206       0.0839
   0.1557    ]
 [-0.0857      0.0475      0.14489999 ...  0.0586      0.11
   0.0743    ]]


In [9]:
#2.4 Text / NLP based features
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [10]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [11]:
#2.5 Topic Models as features
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))


In [22]:
#3. Model Building
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [13]:
#3.1 Naive Bayes
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.8356
NB, WordLevel TF-IDF:  0.8472
NB, N-Gram Vectors:  0.8328
NB, CharLevel Vectors:  0.814


In [21]:
accuracy = train_model(naive_bayes.ComplementNB(), xtrain_count, train_y, xvalid_count)
print ("CB, Count Vectors: ", accuracy)

AttributeError: module 'sklearn.naive_bayes' has no attribute 'ComplementNB'

In [13]:
NB, Count Vectors:  0.8488
NB, WordLevel TF-IDF:  0.8524
NB, N-Gram Vectors:  0.8448
NB, CharLevel Vectors:  0.8108

SyntaxError: invalid syntax (<ipython-input-13-166b17a58bdd>, line 1)

In [14]:
#3.2 Linear Classifier
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.864
LR, WordLevel TF-IDF:  0.8708
LR, N-Gram Vectors:  0.8364
LR, CharLevel Vectors:  0.8476


In [ ]:
LR, Count Vectors:  0.8688
LR, WordLevel TF-IDF:  0.872
LR, N-Gram Vectors:  0.8364
LR, CharLevel Vectors:  0.8448

In [28]:
#3. Model Building
def train_model_SVM(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)    
    return metrics.accuracy_score(predictions, valid_y)

In [29]:
accuracy = train_model_SVM(svm.SVC(kernel='linear'), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.8424


In [27]:
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(xtrain_count, train_y)
y_pred = clf.predict(xvalid_count)
print("Accuracy:",metrics.accuracy_score(valid_y, y_pred))

Accuracy: 0.8528


In [30]:
accuracy = train_model_SVM(svm.SVC(kernel='linear'), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM,WordLevel TF-IDF: ", accuracy)
accuracy = train_model_SVM(svm.SVC(kernel='linear'), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM,CharLevel Vectors: ", accuracy)

SVM,WordLevel TF-IDF:  0.878
SVM,CharLevel Vectors:  0.8556


In [ ]:
SVM, N-Gram Vectors:  0.8424
SVM,WordLevel TF-IDF:  0.878
SVM,CharLevel Vectors:  0.8556

In [17]:
#3. Model Building
def train_model_SVC(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)    
    return metrics.accuracy_score(predictions, valid_y)

In [18]:
#3.3 Implementing a SVM Model
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model_SVC(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.84


In [20]:
accuracy = train_model_SVC(svm.SVC(C=100.0, gamma= 0.001), xtrain_count, train_y, xvalid_count)
print ("SVM,Count Vectors: ", accuracy)

SVM,Count Vectors:  0.856


In [21]:
accuracy = train_model_SVC(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM,WordLevel TF-IDF: ", accuracy)
accuracy = train_model_SVC(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM,CharLevel Vectors: ", accuracy)

SVM,WordLevel TF-IDF:  0.8704
SVM,CharLevel Vectors:  0.836


In [ ]:
SVM, N-Gram Vectors:  0.84
SVM,Count Vectors:  0.856
SVM,WordLevel TF-IDF:  0.8704
SVM,CharLevel Vectors:  0.836

In [27]:
#3.4 Bagging Model
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100,criterion='entropy'), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)
# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100,criterion='entropy'), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.8344
RF, WordLevel TF-IDF:  0.8344


In [ ]:
RF, Count Vectors:  0.8464
RF, WordLevel TF-IDF:  0.8344

In [30]:
#3.5 Boosting Model
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=500,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier( learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, Count Vectors:  0.8584


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, WordLevel TF-IDF:  0.8356
Xgb, CharLevel Vectors:  0.8132


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
Xgb, Count Vectors: 0.8584
Xgb, WordLevel TF-IDF: 0.8356
Xgb, CharLevel Vectors:  0.8152

In [43]:
#3. Model Building
def train_model_new(classifier, feature_vector_train, label, feature_vector_valid, epoch, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label,epochs=epoch)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    #print(predictions.shape)
    print(predictions[0:20])
   
    predictions = [1 if x >= 0.2 else 0 for x in predictions[:, 0]]
    #if is_neural_net:
        #predictions = predictions.argmax(axis=1)
    
    print(predictions[0:20])
    print(valid_y[0:20])
    #print(metrics.accuracy_score(valid_y,predictions))
    return metrics.roc_auc_score(valid_y,predictions) 

In [44]:
#3.6 Shallow Neural Networks
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model_new(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 20, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/20
7500/7500 [==============================] - 4s 532us/step - loss: 0.5257 - acc: 0.7780
Epoch 2/20
7500/7500 [==============================] - 4s 487us/step - loss: 0.2806 - acc: 0.8913
Epoch 3/20
7500/7500 [==============================] - 4s 499us/step - loss: 0.1967 - acc: 0.9304
Epoch 4/20
7500/7500 [==============================] - 4s 486us/step - loss: 0.1456 - acc: 0.9535
Epoch 5/20
7500/7500 [==============================] - 4s 504us/step - loss: 0.1077 - acc: 0.9699
Epoch 6/20
7500/7500 [==============================] - 4s 489us/step - loss: 0.0795 - acc: 0.9804
Epoch 7/20
7500/7500 [==============================] - 4s 530us/step - loss: 0.0576 - acc: 0.9904
Epoch 8/20
7500/7500 [==============================] - 4s 498us/step - loss: 0.0392 - acc: 0.9969
Epoch 9/20
7500/7500 [==============================] - 4s 472us/step - loss: 0.0281 - acc: 0.9984
Epoch 10/20
7500/7500 [==============================] - 4s 479us/step - loss: 0.0202 - acc: 0.9988
Epoch 11/

In [ ]:
Epoch 1/1
7500/7500 [==============================] - 8s 1ms/step - loss: 0.5251
NN, Ngram Level TF IDF Vectors 0.5016

Epoch 1/20
7500/7500 [==============================] - 5s 709us/step - loss: 0.5273 - acc: 0.7760
Epoch 2/20
7500/7500 [==============================] - 4s 470us/step - loss: 0.2785 - acc: 0.8925
Epoch 3/20
7500/7500 [==============================] - 4s 487us/step - loss: 0.1936 - acc: 0.9320
Epoch 4/20
7500/7500 [==============================] - 3s 390us/step - loss: 0.1423 - acc: 0.9543
Epoch 5/20
7500/7500 [==============================] - 3s 385us/step - loss: 0.1046 - acc: 0.9719
Epoch 6/20
7500/7500 [==============================] - 3s 382us/step - loss: 0.0752 - acc: 0.9845
Epoch 7/20
7500/7500 [==============================] - 3s 385us/step - loss: 0.0527 - acc: 0.9931
Epoch 8/20
7500/7500 [==============================] - 3s 382us/step - loss: 0.0368 - acc: 0.9968
Epoch 9/20
7500/7500 [==============================] - 3s 381us/step - loss: 0.0259 - acc: 0.9979
Epoch 10/20
7500/7500 [==============================] - 3s 447us/step - loss: 0.0188 - acc: 0.9987
Epoch 11/20
7500/7500 [==============================] - 3s 466us/step - loss: 0.0141 - acc: 0.9987
Epoch 12/20
7500/7500 [==============================] - 4s 517us/step - loss: 0.0111 - acc: 0.9988
Epoch 13/20
7500/7500 [==============================] - 5s 624us/step - loss: 0.0090 - acc: 0.9988
Epoch 14/20
7500/7500 [==============================] - 4s 472us/step - loss: 0.0075 - acc: 0.9988
Epoch 15/20
7500/7500 [==============================] - 4s 473us/step - loss: 0.0064 - acc: 0.9988
Epoch 16/20
7500/7500 [==============================] - 4s 474us/step - loss: 0.0056 - acc: 0.9988
Epoch 17/20
7500/7500 [==============================] - 4s 480us/step - loss: 0.0050 - acc: 0.9988
Epoch 18/20
7500/7500 [==============================] - 4s 476us/step - loss: 0.0045 - acc: 0.9987
Epoch 19/20
7500/7500 [==============================] - 4s 478us/step - loss: 0.0041 - acc: 0.9987
Epoch 20/20
7500/7500 [==============================] - 4s 478us/step - loss: 0.0038 - acc: 0.9987
NN, Ngram Level TF IDF Vectors 0.8967110726366244

In [45]:
#3.7 Deep Neural Networks
#3.7.1 Convolutional Neural Network
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

classifier = create_cnn()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 20, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

NameError: name 'word_index' is not defined

In [ ]:
Epoch 1/1
7500/7500 [==============================] - 17s 2ms/step - loss: 0.5841
CNN, Word Embeddings 0.5016

Epoch 1/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.5549
Epoch 2/20
7500/7500 [==============================] - 12s 2ms/step - loss: 0.3796
Epoch 3/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.3206
Epoch 4/20
7500/7500 [==============================] - 12s 2ms/step - loss: 0.2778
Epoch 5/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.2384
Epoch 6/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.1917
Epoch 7/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.1575
Epoch 8/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.1238
Epoch 9/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.1015
Epoch 10/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0889
Epoch 11/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0688
Epoch 12/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0716
Epoch 13/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0612
Epoch 14/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.0563
Epoch 15/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0523
Epoch 16/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.0483
Epoch 17/20
7500/7500 [==============================] - 13s 2ms/step - loss: 0.0537
Epoch 18/20
7500/7500 [==============================] - 15s 2ms/step - loss: 0.0442
Epoch 19/20
7500/7500 [==============================] - 14s 2ms/step - loss: 0.0456
Epoch 20/20
7500/7500 [==============================] - 12s 2ms/step - loss: 0.0425
(2500, 1)
[7.8318059e-01 9.9925035e-01 6.1134610e-04 9.9998665e-01 9.9999905e-01
 9.9962318e-01 9.9859709e-01 9.9917930e-01 7.0752937e-01 9.9864143e-01
 9.6448255e-01 1.1390647e-04 9.6624172e-01 5.3794348e-01 9.9877435e-01
 6.1611372e-01 9.9974352e-01 1.3516644e-06 8.7371105e-01 7.7482120e-08]
[1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 1 0]
CNN, Word Embeddings 0.9373635358053192


In [22]:
#3.7.2 Recurrent Neural Network – LSTM
# Add an Input Layer
def create_rnn_lstm():
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 20, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/20
7500/7500 [==============================] - 30s 4ms/step - loss: 0.5878 - acc: 0.6832
Epoch 2/20
7500/7500 [==============================] - 33s 4ms/step - loss: 0.5014 - acc: 0.7619
Epoch 3/20
7500/7500 [==============================] - 32s 4ms/step - loss: 0.4734 - acc: 0.7832
Epoch 4/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4569 - acc: 0.7856
Epoch 5/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4377 - acc: 0.7977
Epoch 6/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4178 - acc: 0.8145
Epoch 7/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3922 - acc: 0.8229
Epoch 8/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3734 - acc: 0.8315
Epoch 9/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3548 - acc: 0.8428
Epoch 10/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3319 - acc: 0.8535
Epoch 11/20
7500/75

In [ ]:
Epoch 1/20
7500/7500 [==============================] - 30s 4ms/step - loss: 0.5878 - acc: 0.6832
Epoch 2/20
7500/7500 [==============================] - 33s 4ms/step - loss: 0.5014 - acc: 0.7619
Epoch 3/20
7500/7500 [==============================] - 32s 4ms/step - loss: 0.4734 - acc: 0.7832
Epoch 4/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4569 - acc: 0.7856
Epoch 5/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4377 - acc: 0.7977
Epoch 6/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.4178 - acc: 0.8145
Epoch 7/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3922 - acc: 0.8229
Epoch 8/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3734 - acc: 0.8315
Epoch 9/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3548 - acc: 0.8428
Epoch 10/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3319 - acc: 0.8535
Epoch 11/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3195 - acc: 0.8592
Epoch 12/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.3079 - acc: 0.8683
Epoch 13/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2907 - acc: 0.8735
Epoch 14/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2714 - acc: 0.8859
Epoch 15/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2528 - acc: 0.8923
Epoch 16/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2457 - acc: 0.8949
Epoch 17/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2277 - acc: 0.9029
Epoch 18/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2167 - acc: 0.9093
Epoch 19/20
7500/7500 [==============================] - 27s 4ms/step - loss: 0.2068 - acc: 0.9111
Epoch 20/20
7500/7500 [==============================] - 28s 4ms/step - loss: 0.1888 - acc: 0.9235
(2500, 1)
[[5.6749523e-02]
 [8.3308637e-01]
 [9.9879092e-01]
 [9.9999177e-01]
 [2.0139949e-02]
 [3.8266245e-02]
 [5.1640469e-04]
 [5.4006672e-01]
 [4.0028162e-02]
 [3.0131629e-01]
 [4.0146457e-05]
 [9.9299735e-01]
 [9.9577868e-01]
 [2.2251075e-02]
 [9.9274325e-01]
 [9.9896109e-01]
 [9.9785280e-01]
 [9.9829048e-01]
 [2.9750664e-03]
 [5.2361798e-01]]
[0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
[0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0]
RNN-LSTM, Word Embeddings 0.8376

In [23]:
#3.7.3 Recurrent Neural Network – GRU
def create_rnn_gru():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 23s 3ms/step - loss: 0.6119
RNN-GRU, Word Embeddings 0.5


In [ ]:
Epoch 1/1
7500/7500 [==============================] - 93s 12ms/step - loss: 0.6193
RNN-GRU, Word Embeddings 0.5016

In [24]:
#3.7.4 Bidirectional RNN
def create_bidirectional_rnn():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 40s 5ms/step - loss: 0.5891
RNN-Bidirectional, Word Embeddings 0.5


In [ ]:
Epoch 1/1
7500/7500 [==============================] - 63s 8ms/step - loss: 0.6233
RNN-Bidirectional, Word Embeddings 0.5016

In [25]:
#3.7.5 Recurrent Convolutional Neural Network
def create_rcnn():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 15s 2ms/step - loss: 0.5856
CNN, Word Embeddings 0.5


In [ ]:
Epoch 1/1
7500/7500 [==============================] - 16s 2ms/step - loss: 0.5710
CNN, Word Embeddings 0.5016